In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# YOLO v7

In [ ]:
!pip install GPUtil

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  1% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  2% |  1% |


In [ ]:
# Create .yaml file 
import yaml

data_yaml = dict(
    train = '/content/train',
    val = '/content/val',
    nc = 1,
    names = ['Container']
)

# Note that I am creating the file in the yolov5/data/ directory.
with open('/content/data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)

In [ ]:
!git clone https://github.com/WongKinYiu/yolov7

Cloning into 'yolov7'...
remote: Enumerating objects: 1185, done.
remote: Total 1185 (delta 0), reused 0 (delta 0), pack-reused 1185
Receiving objects: 100% (1185/1185), 74.23 MiB | 18.94 MiB/s, done.
Resolving deltas: 100% (509/509), done.


In [ ]:

%cd yolov7

/content/yolov7


In [ ]:
%ls

cfg/       export.py   LICENSE.md  requirements.txt  train_aux.py
data/      figure/     models/     scripts/          train.py
deploy/    hubconf.py  paper/      test.py           utils/
detect.py  inference/  README.md   tools/


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!gdown "https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt"

Downloading...
From: https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt
To: /content/yolov7/yolov7.pt
100% 75.6M/75.6M [00:01<00:00, 59.9MB/s]


In [ ]:
!python train.py --batch 4 --cfg cfg/training/yolov7.yaml --img 768 1024 --epochs 10 --data /content/data.yaml --weights '' --name Model --hyp ./data/hyp.scratch.p5.yaml --device 0

2023-06-01 12:30:01.551398: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 12:30:03.020621: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
YOLOR 🚀 v0.1-126-g84932d7 torch 2.0.1+cu118 CUDA:0 (Tesla T4, 15101.8125MB)

Namespace(weights='', cfg='cfg/training/yolov7.yaml', data='/content/data.yaml', hyp='./data/hyp.scratch.p5.yaml', epochs=10, batch_size=4, img_size=[768, 1024], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='0', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=8, project='runs/train', entity=None, name='Model', exist_ok=False, quad=False, linear

In [ ]:
!python detect.py --weights /content/yolov7/runs/train/Model3/weights/best.pt --conf 0.3 --source /content/val

Namespace(weights=['/content/yolov7/runs/train/Model3/weights/best.pt'], source='/content/val', img_size=640, conf_thres=0.3, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 v0.1-126-g84932d7 torch 2.0.1+cu118 CUDA:0 (Tesla T4, 15101.8125MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36481772 parameters, 6194944 gradients, 103.2 GFLOPS
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is t

In [ ]:
!darknet detector map data/obj.data /content/yolov7/cfg/training/yolov7.yaml backup/yolo-obj_final.weights 

In [ ]:
import glob
from IPython.display import Image, display

i = 0
limit = 10000 # max images to print
for imageName in glob.glob('/content/yolov7/runs/detect/exp2/*.jpg'): #assuming JPG  
    if i < limit:
      display(Image(filename=imageName))
      print("\n")
    i = i + 1

In [ ]:
import cv2
import json
import os

data_dir = "/content/yolov7/test"
result_file = "/content/yolov7/runs/train/Model3/results.txt"
output_dir = "/content/yolov7/runs/train/Model3/output/"

# 檢查輸出目錄是否存在，如果不存在則建立目錄
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 讀取結果檔案
with open(result_file, 'r') as f:
    results = json.load(f)

# 迭代處理每個結果
for result in results:
    filename = os.path.basename(result['filename'].strip())  # 取得不包含路徑的檔案名稱
    objects = result['objects']

    # 讀取原始影像
    image_path = os.path.join(data_dir, filename)
    image = cv2.imread(image_path)

    # 檢查影像是否成功讀取
    if image is None:
        print(f"無法讀取影像：{image_path}")
        continue

    # 迭代處理每個物件
    for obj in objects:
        class_id = obj['class_id']
        center_x = obj['relative_coordinates']['center_x']
        center_y = obj['relative_coordinates']['center_y']
        width = obj['relative_coordinates']['width']
        height = obj['relative_coordinates']['height']

        # 計算物件在影像中的位置
        left = int((center_x - width/2) * image.shape[1])
        top = int((center_y - height/2) * image.shape[0])
        right = int((center_x + width/2) * image.shape[1])
        bottom = int((center_y + height/2) * image.shape[0])

        # 切割出物件影像
        object_image = image[top:bottom, left:right]

        # 儲存物件影像
        object_filename = os.path.splitext(filename)[0] + "_" + str(class_id) + ".jpg"
        save_path = os.path.join(output_dir, object_filename)
        print(save_path)
        cv2.imwrite(save_path, object_image)

In [ ]:
data_dir = "/content/yolov7/test"
result_file = "/content/yolov7/runs/train/Model3/results.txt"
output_dir = "/content/yolov7/runs/train/Model3/output/"

# 檢查輸出目錄是否存在，如果不存在則建立目錄
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 讀取結果檔案
with open(result_file, 'r') as f:
    results = json.load(f)

# 迭代處理每個結果
for result in results:
    filename = os.path.basename(result['filename'].strip())  # 取得不包含路徑的檔案名稱
    objects = result['objects']

    # 讀取原始影像
    image_path = os.path.join(data_dir, filename)
    image = cv2.imread(image_path)

    # 檢查影像是否成功讀取
    if image is None:
        print(f"無法讀取影像：{image_path}")
        continue

    # 迭代處理每個物件
    for obj in objects:
        class_id = obj['class_id']
        center_x = obj['relative_coordinates']['center_x']
        center_y = obj['relative_coordinates']['center_y']
        width = obj['relative_coordinates']['width']
        height = obj['relative_coordinates']['height']

        # 計算物件在影像中的位置
        left = int((center_x - width/2) * image.shape[1])
        top = int((center_y - height/2) * image.shape[0])
        right = int((center_x + width/2) * image.shape[1])
        bottom = int((center_y + height/2) * image.shape[0])

        # 切割出物件影像
        object_image = image[top:bottom, left:right]

        # 檢查切割的物件影像是否為空
        if object_image.size == 0:
            print(f"空的物件影像：{save_path}")
            continue

        # 儲存物件影像
        object_filename = os.path.splitext(filename)[0] + ".jpg"
        save_path = os.path.join(output_dir, object_filename)
        cv2.imwrite(save_path, object_image)

In [ ]:
# this needs to run only once to load the model into memory
reader = easyocr.Reader(['en'])

In [ ]:
directory = 'data/output/'

# 取得目錄中的所有文件
files = os.listdir(directory)

# 遍歷每個文件
for file_name in files:
    # 構建文件的完整路徑
    file_path = os.path.join(directory, file_name)
    
    # 檢查是否為檔案而非目錄
    if os.path.isfile(file_path):
        # 進行您的處理
        result = reader.readtext(file_path, detail=0)
        
        # 將結果進行字串處理
        processed_result = ' '.join(result).replace(' ', '')
        
        # 只保留11個字元
        processed_result = processed_result[:11]
        
        # 輸出處理後的結果
        print(file_path, processed_result)

In [ ]:
directory = 'data/output_35/'

# 取得目錄中的所有文件
files = os.listdir(directory)

# 遍歷每個文件
for file_name in files:
    # 構建文件的完整路徑
    file_path = os.path.join(directory, file_name)
    
    # 檢查是否為檔案而非目錄
    if os.path.isfile(file_path):
        # 進行您的處理
        result = reader.readtext(file_path, detail=0)
        
        # 將結果進行字串處理
        processed_result = ' '.join(result).replace(' ', '')
        
        # 只保留11個字元
        processed_result = processed_result[:11]
        
        # 輸出處理後的結果
        print(file_path, processed_result)